# purpose : New pretrained file for DeepgazeIIE model 

## data & frame 준비 

In [1]:
# data path
base_path = '/Users/krc/Documents/retail/'

In [2]:
# install check 
import torch
import os
import csv
torch.__version__

'1.13.0.dev20220623'

In [3]:
os.getcwd()

'/Users/krc/Documents/retail/retail_gh/deepgaze_sample'

In [4]:
%cd ../../

/Users/krc/Documents/retail


In [5]:
# #mmdetection installation 
# !pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
# !pip install -r requirements/build.txt
# !pip install -v -e .
# !pip install mmdet

In [6]:
os.listdir()

['1조_리테일앤인사이트_기획안발표.pptx',
 '리테일앤인사이트_RFP.docx',
 '.DS_Store',
 'DeepGazeIIE-1.0.0',
 'retail_gh',
 'work',
 'mmdetection',
 'deep_gaze_jup',
 '1조_리테일앤인사이트_기획안발표_수정1.pptx',
 'data',
 'reference']

In [7]:
# annotation by csv 
f =open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/annotations_train.csv','r')
annotation = csv.reader(f)
# image_name,x1,y1,x2,y2,class,image_width,image_height
ls_ann = list(annotation)
print(ls_ann)
f.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
index = ['image_name','x1','y1','x2','y2','class','image_width','image_height']
print(index)
print(ls_ann[-1])
print(ls_ann[-1][0])


['image_name', 'x1', 'y1', 'x2', 'y2', 'class', 'image_width', 'image_height']
['train_999.jpg', '905', '1589', '1144', '1712', 'object', '2336', '4160']
train_999.jpg


In [9]:
# annotation 생성
import copy
import os.path as osp
import cv2
import yaml

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

CLASSES = ('object')
# cat2label = {k:i for i, k in enumerate(CLASSES)}

# 반드시 아래 Decorator 설정 할것.@DATASETS.register_module() 설정 시 force=True를 입력하지 않으면 Dataset 재등록 불가. 
# @DATASETS.register_module(force=True)
# class SKIKU110K(CustomDataset):
#   CLASSES = ('object')
  
  ##### self.data_root: ./kitti_tiny/ self.ann_file: ./kitti_tiny/train.txt self.img_prefix: ./kitti_tiny/training/image_2
  #### ann_file: ./kitti_tiny/train.txt
  # annotation에 대한 모든 파일명을 가지고 있는 텍스트 파일을 __init__(self, ann_file)로 입력 받고, 이 self.ann_file이 load_annotations()의 인자로 입력
def load_annotations(self, ann_file):
#     print('##### self.data_root:', self.data_root, 'self.ann_file:', self.ann_file, 'self.img_prefix:', self.img_prefix)
#     print('#### ann_file:', ann_file)
#    cat2label = {k:i for i, k in enumerate(self.CLASSES)}
    # 포맷 중립 데이터를 담을 list 객체
    data_infos = []
    
    for label in ann_file:
      filename = label[0]
      height, width = label[6],label[7]
      # 개별 image의 annotation 정보 저장용 Dict 생성. key값 filename 에는 image의 파일명만 들어감(디렉토리는 제외)
      data_info = {'filename': str(filename),
                   'width': int(width), 'height': int(height)}

      # 오브젝트의 클래스명은 bbox_names로 저장. 
      bbox_names = label[5]
      # bbox 좌표를 저장
      bboxes = [int(label[1]),int(label[2]), int(label[3]), int(label[4])]

#       # 클래스명이 해당 사항이 없는 대상 Filtering out, 'DontCare'sms ignore로 별도 저장.
#       gt_bboxes = []
#       gt_labels = []
#       gt_bboxes_ignore = []
#       gt_labels_ignore = []

#       for bbox_name, bbox in zip(bbox_names, bboxes):
#         # 만약 bbox_name이 클래스명에 해당 되면, gt_bboxes와 gt_labels에 추가, 그렇지 않으면 gt_bboxes_ignore, gt_labels_ignore에 추가
#         if bbox_name in cat2label:
#           gt_bboxes.append(bbox)
#           # gt_labels에는 class id를 입력
#           gt_labels.append(cat2label[bbox_name])
#         else:
#           gt_bboxes_ignore.append(bbox)
#           gt_labels_ignore.append(-1)
#       # 개별 image별 annotation 정보를 가지는 Dict 생성. 해당 Dict의 value값은 모두 np.array임. 
      data_anno = {
          'bboxes': [ bboxes ],
          'labels': [(0,)], #gt_labels
          'bboxes_ignore': np.zeros((1, 4), dtype=np.float32).tolist(),
          'labels_ignore': np.zeros((1, 4), dtype=np.float32).tolist()
      }
      # image에 대한 메타 정보를 가지는 data_info Dict에 'ann' key값으로 data_anno를 value로 저장. 
      data_info.update(ann=data_anno)
      # 전체 annotation 파일들에 대한 정보를 가지는 data_infos에 data_info Dict를 추가
      data_infos.append(data_info)

    return data_infos

In [10]:

# new_ann = load_annotations(CustomDataset,ann_file=ls_ann)
# print(new_ann)

In [11]:
# import json
 
# with open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/train_ann.json', 'w') as f:
#     json.dump(new_ann, f,indent=4)

In [12]:
# # annotation by csv 
# f =open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/annotations_val.csv','r')
# annotation = csv.reader(f)
# # image_name,x1,y1,x2,y2,class,image_width,image_height
# val_ann = list(annotation)
# f.close()

# import json
# val_ann = load_annotations(CustomDataset,ann_file=val_ann)


# with open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/val_ann.json', 'w') as f:
#     json.dump(val_ann, f,indent=4)

In [13]:
os.getcwd()

'/Users/krc/Documents/retail'

## DETR 사전 가중치 다운로드 

In [14]:
os.chdir('/Users/krc/Documents/retail/mmdetection/')
if not os.path.isdir("checkpoint"):
    os.mkdir("checkpoint")
os.chdir('checkpoint')

from requests import get 

def download(url, file_name):
    with open(file_name, "wb") as file:   
        response = get(url)               
        file.write(response.content)


# url = "https://github.com/open-mmlab/mmdetection/tree/master/configs/deformable_detr"
# download(url,"deformable_detr_r50_16x2_50e_coco_20210419_220030-a12b9512.pth.pth")

os.chdir('/Users/krc/Documents/retail/mmdetection/')

## wand (굳이..) 

In [15]:
#wandb setting first 
import wandb

In [16]:
wandb.login()

wandb: Currently logged in as: rumj. Use `wandb login --relogin` to force relogin


True

In [23]:
#!pip install mmcv-full
#!pip install terminaltables
#!pip install imagecorruptions
#!pip install -q pycocotools

import mmcv
from mmcv.runner import load_checkpoint
from mmdet.apis import inference_detector, show_result_pyplot
from mmdet.models import build_detector
from mmdet.apis import set_random_seed
import os.path as osp

def set_config():
  # 사용할 모델 초기 py 설정
  config = '/Users/krc/Documents/retail/mmdetection/configs/deformable_detr/deformable_detr_r50_16x2_50e_coco.py'

  # config 불러오기
  cfg = mmcv.Config.fromfile(config)

  # config 에 base 기록
  cfg.base = '/Users/krc/Documents/retail/mmdetection/configs/detr/detr_r50_8x2_150e_coco.py'

  # 데이터셋 지정
  cfg.dataset_type = 'CustomDataset'

  # data_root는 사용자가 저장한 데이터가 있는 폴더 전까지
  data_root = '/Users/krc/Documents/retail/data/SKU110K_fixed/'

  # class 설정
  cfg.classes = ('object',)

  # train, val, test 데이터셋에 대한 type, data_root, ann_file, img_prefix 설정

  cfg.data.samples_per_gpu=1
  cfg.data.workers_per_gpu=0

  cfg.data.train.data_root = data_root
  cfg.data.train.ann_file = '/Users/krc/Documents/retail/data/SKU110K_fixed/annotations_train.json/annotations_train.json' 
  cfg.data.train.img_prefix = 'train/' 
  cfg.data.train.classes = cfg.classes
#  cfg.data.train.type = 'CustomDataset'

  cfg.data.val.data_root = data_root 
  cfg.data.val.ann_file = '/Users/krc/Documents/retail/data/SKU110K_fixed/annotations_val.json/annotations_val.json'
  cfg.data.val.img_prefix = 'val/'
  cfg.data.val.classes = cfg.classes
#  cfg.data.val.type = 'CustomDataset'

  cfg.data.test.data_root = data_root 
  cfg.data.test.ann_file = 'test.json'
  cfg.data.test.img_prefix = 'test/'
  cfg.data.test.classes = cfg.classes
#  cfg.data.test.type = 'CustomDataset'

  # 클래스 수 지정
  cfg.model.bbox_head.num_classes = 1
#  cfg.model.roi_head.mask_head.num_classes = 1


  # 사전 훈련 모델 지정
  cfg.checkpoint_config = dict(interval=10,out_dir='work/ch_config/')
  cfg.load_from = '/Users/krc/Documents/retail/mmdetection/checkpoint/deformable_detr_r50_16x2_50e_coco_20210419_220030-a12b9512.pth'


  # 가중치 저장 위치
  cfg.work_dir =  './work/weights/'

  # 평가 지표로 설정
  cfg.evaluation.metric = ['bbox']

  # epoch 설정
  cfg.runner = dict(type='EpochBasedRunner', max_epochs=40) 

  # batch size 설정
  cfg.auto_scale_lr = dict(enable=False, base_batch_size=32)

  # gpu 설정 및 seed 설정
  cfg.seed = 42
  cfg.gpu_ids = range(1)
  cfg.device= 'cpu' # for local

  #wandb setting
  cfg.log_config.hooks = [
        dict(type='TextLoggerHook'),
        dict(type='MMDetWandbHook',
             init_kwargs={
                          'project': 'SKU_DETR',
                          'config': {"optimizer" : 'AdamW, default',
                          "batch_size" :32,
                          "epochs" :40,
                          "lr_config" : 'CosineAnnealning' }
                         },
             interval=4,
             log_checkpoint=True,
             log_checkpoint_metadata=True,
             num_eval_images=100,
             bbox_score_thr=0.1)]

  #learning rate
  cfg.lr_config = dict(
   policy='CosineAnnealing',
   by_epoch=False,
   min_lr=0)
    
    
  meta = dict()
  meta['exp_name'] = osp.basename(config)
  print(meta)


  return cfg



In [24]:
cfg = set_config()

{'exp_name': 'deformable_detr_r50_16x2_50e_coco.py'}


In [25]:
print(cfg.pretty_text)

dataset_type = 'CustomDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='AutoAugment',
        policies=[[{
            'type':
            'Resize',
            'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333),
                          (608, 1333), (640, 1333), (672, 1333), (704, 1333),
                          (736, 1333), (768, 1333), (800, 1333)],
            'multiscale_mode':
            'value',
            'keep_ratio':
            True
        }],
                  [{
                      'type': 'Resize',
                      'img_scale': [(400, 4200), (500, 4200), (600, 4200)],
                      'multiscale_mode': 'value',
                      'keep_ratio': True
                  }, {
              

## train

In [26]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# dataset 생성 및 model 설정

datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model)
model.CLASSES = datasets[0].CLASSES
# 훈련
train_detector(model, datasets, cfg, distributed=False, validate=True)



loading annotations into memory...
Done (t=3.57s)
creating index...
index created!


2022-10-12 13:42:02,973 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-10-12 13:42:03,177 - mmdet - INFO - load checkpoint from local path: /Users/krc/Documents/retail/mmdetection/checkpoint/deformable_detr_r50_16x2_50e_coco_20210419_220030-a12b9512.pth


loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


2022-10-12 13:42:03,288 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.cls_branches.0.weight: copying a param with shape torch.Size([80, 256]) from checkpoint, the shape in current model is torch.Size([1, 256]).
size mismatch for bbox_head.cls_branches.0.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([1]).
size mismatch for bbox_head.cls_branches.1.weight: copying a param with shape torch.Size([80, 256]) from checkpoint, the shape in current model is torch.Size([1, 256]).
size mismatch for bbox_head.cls_branches.1.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([1]).
size mismatch for bbox_head.cls_branches.2.weight: copying a param with shape torch.Size([80, 256]) from checkpoint, the shape in current model is torch.Size([1, 256]).
size mismatch for bbox_head.cls_branches.2.bias: copying a param with shape torch

2022-10-12 13:42:06,257 - mmdet - WARNING - No meta information found in the runner. 
Corrupt JPEG data: 305 extraneous bytes before marker 0xd9
wandb: Downloading large artifact val:v0, 853.41MB. 102 files... 
wandb:   102 of 102 files downloaded.  
Done. 0:0:0.5
Corrupt JPEG data: premature end of data segment
2022-10-12 13:50:29,385 - mmdet - INFO - Epoch [1][50/8219]	lr: 2.000e-04, eta: 19 days, 0:35:13, time: 5.000, data_time: 0.107, loss_cls: 0.9752, loss_bbox: 0.0000, loss_iou: 0.0000, d0.loss_cls: 3.7520, d0.loss_bbox: 0.0000, d0.loss_iou: 0.0000, d1.loss_cls: 2.8813, d1.loss_bbox: 0.0000, d1.loss_iou: 0.0000, d2.loss_cls: 1.8678, d2.loss_bbox: 0.0000, d2.loss_iou: 0.0000, d3.loss_cls: 1.4603, d3.loss_bbox: 0.0000, d3.loss_iou: 0.0000, d4.loss_cls: 1.2569, d4.loss_bbox: 0.0000, d4.loss_iou: 0.0000, loss: 12.1935, grad_norm: 313.0049
2022-10-12 13:56:22,361 - mmdet - INFO - Epoch [1][100/8219]	lr: 2.000e-04, eta: 22 days, 22:30:21, time: 7.060, data_time: 0.086, loss_cls: 0.0000

...

Corrupt JPEG data: premature end of data segment


[>>>>>>>>>>>>>>>             ] 319/588, 0.1 task/s, elapsed: 3149s, ETA:  2656s

Corrupt JPEG data: 366 extraneous bytes before marker 0xd9


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 588/588, 0.2 task/s, elapsed: 3884s, ETA:     0s

IndexError: list index out of range

In [ ]:
# visulization 

img = mmcv.imread(mm_path+'/data/dataset/test/0088.jpg')

model.cfg = cfg
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

## model test & out json 

In [22]:
cfg = set_config()

{'exp_name': 'deformable_detr_r50_16x2_50e_coco.py'}


In [ ]:
import argparse
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)

from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.utils import (build_ddp, build_dp, compat_cfg, get_device,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)





# 필요한 파라미터 설정
checkpoint='work_dirs/lg_mask/lg_0801/latest.pth' # 훈련시킨 모델 가중치
show=False
show_dir=None
show_score_thr=0.3
eval_options={'jsonfile_prefix': './work_dirs/out_json/predict'} # json 출력 저장위치
format_only = True
out = None





## 여기서 부터 기존의 mmdetection/tools/test.py 를 변경한 코드 ##

cfg = replace_cfg_vals(cfg)
update_data_root(cfg)
cfg = compat_cfg(cfg)
setup_multi_processes(cfg)



# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

if 'pretrained' in cfg.model:
    cfg.model.pretrained = None
elif 'init_cfg' in cfg.model.backbone:
    cfg.model.backbone.init_cfg = None

if cfg.model.get('neck'):
    if isinstance(cfg.model.neck, list):
        for neck_cfg in cfg.model.neck:
            if neck_cfg.get('rfp_backbone'):
                if neck_cfg.rfp_backbone.get('pretrained'):
                    neck_cfg.rfp_backbone.pretrained = None
    elif cfg.model.neck.get('rfp_backbone'):
        if cfg.model.neck.rfp_backbone.get('pretrained'):
            cfg.model.neck.rfp_backbone.pretrained = None



cfg.gpu_ids = [cfg.gpu_ids]
cfg.device = get_device()


distributed = False


test_dataloader_default_args = dict(
    samples_per_gpu=1, workers_per_gpu=2, dist=distributed, shuffle=False)

# in case the test dataset is concatenated
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(
            cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

test_loader_cfg = {
    **test_dataloader_default_args,
    **cfg.data.get('test_dataloader', {})
}

rank, _ = get_dist_info()


# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, **test_loader_cfg)

# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))

fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')


if 'CLASSES' in checkpoint.get('meta', {}):
    model.CLASSES = checkpoint['meta']['CLASSES']
else:
    model.CLASSES = dataset.CLASSES

if not distributed:
    model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
    outputs = single_gpu_test(model, data_loader, show, show_dir,
                              show_score_thr)


rank, _ = get_dist_info()
if rank == 0:
    if out:
        print(f'\nwriting results to {out}')
        mmcv.dump(outputs, out)
    kwargs = {} if eval_options is None else eval_options
    if format_only:
        dataset.format_results(outputs, **kwargs)

In [ ]:
import json
from datetime import datetime

with open(mm_path+f"/work_dirs/out_json/predict.bbox.json") as f:
    out_data1 = json.load(f)

with open(mm_path+ f"/work_dirs/out_json/predict.segm.json") as f:
    out_data2 = json.load(f)